# MUMBAI OR PUNE? 
## Let's decide!!

### INTRODUCTION

   Mumbai and Pune are the two cities with most importance in Maharashtra,India. There has been a war for supremacy in terms of quality of life, jobs, education, entertainment and recreational facilities that these cities have to offer to its residents. I will like to elaborate in this project that will make an attempt to analyze the neighborhood and give the suggestion to decide between these two cities based on their preference in life.
   
   People living in the respective cities are extremely loyal and proud of their city. One of their favorite hobbies is to debate over which city exceeds and is better and being frank I one of these people.  If you have friends living in these cities, you will always see them argue on which city is better. Trust me it’s a delight to watch them debate. Mumbai has beaches, Pune has Forts. Mumbai is hardworking but people in pune like to live a simple life. Roads in Pune are complicated to understand than mumbai and many more points these people have for debate. But, after experiencing so many debates on this topic I will like to analyse which city is better and which is not.   
   
   Many people from Maharashtra want to settle their. But, they always get confused between where to go? I assure that this Data Science project will help them to decide between these two cities. I'm going to consider top venues in both the cities for comparison. This top venues will include General stores, Restaurants, Bus stops, Ice cream parlours, Coffee shops and many more things near to neighbourhoods in both the cities. Neighbourhoods are different places in the cities.
   The decision to choose one over the other would depend on top venues in the neighborhoods in each of these cities.

### Objective

1. To obtain latitude & longitude of Mumbai and Pune using Nominatim library in Geopy.geocoder.
2. To plot the map of Mumbai and Pune using Folium library.
3. To find out Top venues of each Neibourhood of Mumbai and Pune by using Foursquare API.
4. To cluster the neibourhoods of Mumbai and Pune by K means clustering.
5. To compare both the cities based on top venues.

### Data:-

I'm using the data published by the government of India namely Postal codes of India. Data contents information of office name, pin-code, office type, delivery status, division name, region name, circle name, Taluka, district, state name, related sub office, related head office, latitude and longitude. I will erase unnecessary columns from the data and the final data will consist of office names as neighborhoods only. Neighborhood names with same pin-code will be combined as a single row. As the columns latitude and longitude contains NaN I will be using Foursquare API for obtaining latitude and longitude of each neighborhood. So finally from all the data I will be using all neighborhoods (office names) of Mumbai and Pune.

I am going to use Nominatim library from geocoders.geopy package to find out longitude and latitude for each of the neighborhoods of Mumbai and Pune.
This will give me the data with all required information.

Here, I am providing the link of the data.


https://data.gov.in/resources/all-india-pincode-directory-contact-details-along-latitude-and-longitude


### Methodology

1. Data Cleaning:- In this step unnecessary columns from the data set will be removed that is columns with names office type, delivery status,                 division name, region name, circle name, Taluka, district, state name, related sub office, related head office, latitude and longitude. And           Office names will be named as Neighbourhoods. Neibourhoods(Office names) with same Pincode will be combined.
2. Exploratory Analysis:- Maps of Mumbai and Pune will be plotted by using Folium to visualise the cities. 
3. Data anlysis:- K means clustering technique is used to make the clusters of Neibourhoods.

### Techniques used:-
  1. Kmeans cluster analysis

### Machine learning tools
  1. Python:-
     1. Folium library
     2. Pandas library
     3. Numpy library
     4. Geopy library
     5. JSON library
  2. Foursquare API

#### Importing Required Libraries and data set

In [1]:
import pandas as pd
import numpy as np

In [2]:
# The code was removed by Watson Studio for sharing.

,Neibourhood,Pincode,Districtname
0,Antop Hill,400037,Mumbai
1,B P T Colony,400037,Mumbai
2,B.P.Lane,400003,Mumbai
3,BEST STaff Quarters,400012,Mumbai
4,C G S Colony,400037,Mumbai


#### Combining the rows having the same Pincode

In [3]:
df1=df.groupby('Pincode').agg({
    'Neibourhood': lambda x: ', '.join(x),
    'Districtname': lambda x: x.max()
})
df1.head()

,Neibourhood,Districtname
Pincode,,
400001,"Bazargate , Elephanta Caves Po , M.P.T. , Stoc...",Raigarh(MH)
400002,"Kalbadevi , Ramwadi , S. C. Court , Thakurdwar",Mumbai
400003,"B.P.Lane, Mandvi , Masjid , Null Bazar",Mumbai
400004,"Ambewadi , Charni Road , Chaupati , Girgaon ,...",Mumbai
400005,"Asvini , Colaba Bazar , Colaba , Holiday Camp ...",Mumbai


In [4]:
df1.tail()

,Neibourhood,Districtname
Pincode,,
413130,"Bhigwan , Chincholi , Khadaki , Madanwadi , Ma...",Pune
413132,"Agoti , Dalaj , Loni Deokar , Nhavi , Palasdeo",Pune
413133,"Baramati MIDC , Katfal , Saval",Pune
413801,"Alegaon , Beam Wirelee Station , Boribyal , Da...",Pune
413802,"Kurkumbh , Midc Kurkumbh",Pune


In [5]:
df1.columns=['Neighbourhood','Districtname']
df1.head()

,Neighbourhood,Districtname
Pincode,,
400001,"Bazargate , Elephanta Caves Po , M.P.T. , Stoc...",Raigarh(MH)
400002,"Kalbadevi , Ramwadi , S. C. Court , Thakurdwar",Mumbai
400003,"B.P.Lane, Mandvi , Masjid , Null Bazar",Mumbai
400004,"Ambewadi , Charni Road , Chaupati , Girgaon ,...",Mumbai
400005,"Asvini , Colaba Bazar , Colaba , Holiday Camp ...",Mumbai


In [6]:
import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

!conda install -c conda-forge geocoder --yes
import geocoder
import time

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


In [7]:
locator = Nominatim(user_agent='ny_explorer')

In [8]:
df1['Neighbourhood']= df1['Neighbourhood'].astype(str) +", "+ df1['Districtname']

#### Obtaining Latitude and Longitude of each Neighbourhood

In [9]:
def get_coordinates(Neighbourhood, timeout=5):
    location = locator.geocode(Neighbourhood, timeout=timeout)
    if not location:
        return None, None
    return location.latitude, location.longitude

In [10]:
address1 = 'Mumbai'
address2 = 'Pune'
location1 = locator.geocode(address1)
location2 = locator.geocode(address2)
m_latitude = location1.latitude
p_latitude = location2.latitude
m_longitude = location1.longitude
p_longitude = location2.longitude
print('The geograpical coordinate of Mumbai are {}, {}.'.format(m_latitude, m_longitude))
print('The geograpical coordinate of Pune are {}, {}.'.format(p_latitude, p_longitude))

The geograpical coordinate of Mumbai are 18.9387711, 72.8353355.
The geograpical coordinate of Pune are 18.521428, 73.8544541.


In [11]:
coordinates = df1['Neighbourhood'].apply(lambda x: pd.Series(get_coordinates(x), index=['Latitude', 'Longitude']))
df1 = pd.concat([df1[:], coordinates[:]], axis="columns")

In [12]:
df1.head()

,Neighbourhood,Districtname,Latitude,Longitude
Pincode,,,,
400001,"Bazargate , Elephanta Caves Po , M.P.T. , Stoc...",Raigarh(MH),NaN,NaN
400002,"Kalbadevi , Ramwadi , S. C. Court , Thakurdwar...",Mumbai,NaN,NaN
400003,"B.P.Lane, Mandvi , Masjid , Null Bazar , Mumbai",Mumbai,NaN,NaN
400004,"Ambewadi , Charni Road , Chaupati , Girgaon ,...",Mumbai,NaN,NaN
400005,"Asvini , Colaba Bazar , Colaba , Holiday Camp ...",Mumbai,NaN,NaN


In [13]:
df1.dropna(inplace=True)

In [14]:
df1.head()

,Neighbourhood,Districtname,Latitude,Longitude
Pincode,,,,
400006,"Malabar Hill , Mumbai",Mumbai,18.958162,72.803366
400013,"Delisle Road , Mumbai",Mumbai,18.993260,72.831012
400015,"Sewri , Mumbai",Mumbai,18.998780,72.854422
400017,"Dharavi Road , Dharavi , Mumbai",Mumbai,19.042495,72.854818
400019,"Matunga , Mumbai",Mumbai,19.027436,72.850147


In [15]:
data_final=df1
data_final.head()

,Neighbourhood,Districtname,Latitude,Longitude
Pincode,,,,
400006,"Malabar Hill , Mumbai",Mumbai,18.958162,72.803366
400013,"Delisle Road , Mumbai",Mumbai,18.993260,72.831012
400015,"Sewri , Mumbai",Mumbai,18.998780,72.854422
400017,"Dharavi Road , Dharavi , Mumbai",Mumbai,19.042495,72.854818
400019,"Matunga , Mumbai",Mumbai,19.027436,72.850147


#### Plotting the Map of Mumbai and Pune City

In [16]:
map_mumbai = folium.Map(
    location=[19.0760,72.8777],
    tiles='cartodbpositron',
    zoom_start=12,
)
data_final.apply(lambda row:folium.CircleMarker(location=[row['Latitude'], row['Longitude']]).add_to(map_mumbai), axis=1)
map_mumbai

In [17]:
map_pune = folium.Map(
    location=[18.5204,73.8567],
    tiles='cartodbpositron',
    zoom_start=12,
)
data_final.apply(lambda row:folium.CircleMarker(location=[row['Latitude'], row['Longitude']]).add_to(map_pune), axis=1)
map_pune

In [18]:
mumbai_data = data_final[data_final['Districtname'] == 'Mumbai'].reset_index(drop=True)
mumbai_data.head()

,Neighbourhood,Districtname,Latitude,Longitude
0,"Malabar Hill , Mumbai",Mumbai,18.958162,72.803366
1,"Delisle Road , Mumbai",Mumbai,18.993260,72.831012
2,"Sewri , Mumbai",Mumbai,18.998780,72.854422
3,"Dharavi Road , Dharavi , Mumbai",Mumbai,19.042495,72.854818
4,"Matunga , Mumbai",Mumbai,19.027436,72.850147


In [19]:
pune_data = data_final[data_final['Districtname'] == 'Pune'].reset_index(drop=True)
pune_data.head()

,Neighbourhood,Districtname,Latitude,Longitude
0,"Khandala , Pune",Pune,18.754847,73.376709
1,"Armament , Bavdhan , Sus , Pune",Pune,18.537781,73.796344
2,"Srpf , Pune",Pune,18.502487,73.905953
3,"Khadakwasla R.S. , Pune",Pune,18.439243,73.775766
4,"C M E , Pune",Pune,18.555492,73.952473


#### Process of Foursquare API calls to obtain Top venues in Mumbai & Pune

In [20]:
CLIENT_ID = 'FR43BQ2IQEU02M0YQFTBEVKJIETDI3OGLAXLFRSNWPLK0X2M' # your Foursquare ID
CLIENT_SECRET = 'EJKACBJGC1VAZLLHZ4SVSJQ4A2UUD4MBGLJEM3I4VENF4SKQ' # your Foursquare Secret
VERSION = '20180604' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: FR43BQ2IQEU02M0YQFTBEVKJIETDI3OGLAXLFRSNWPLK0X2M
CLIENT_SECRET:EJKACBJGC1VAZLLHZ4SVSJQ4A2UUD4MBGLJEM3I4VENF4SKQ


In [21]:
mumbai_data=mumbai_data[['Neighbourhood','Latitude','Longitude']]
mumbai_data.head()

,Neighbourhood,Latitude,Longitude
0,"Malabar Hill , Mumbai",18.958162,72.803366
1,"Delisle Road , Mumbai",18.993260,72.831012
2,"Sewri , Mumbai",18.998780,72.854422
3,"Dharavi Road , Dharavi , Mumbai",19.042495,72.854818
4,"Matunga , Mumbai",19.027436,72.850147


In [22]:
pune_data=pune_data[['Neighbourhood','Latitude','Longitude']]
pune_data.head()

,Neighbourhood,Latitude,Longitude
0,"Khandala , Pune",18.754847,73.376709
1,"Armament , Bavdhan , Sus , Pune",18.537781,73.796344
2,"Srpf , Pune",18.502487,73.905953
3,"Khadakwasla R.S. , Pune",18.439243,73.775766
4,"C M E , Pune",18.555492,73.952473


In [23]:
m_neighborhood_latitude = mumbai_data.loc[1, 'Latitude'] # neighborhood latitude value
m_neighborhood_longitude = mumbai_data.loc[1, 'Longitude'] # neighborhood longitude value

m_neighborhood_name = mumbai_data.loc[1, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(m_neighborhood_name, 
                                                               m_neighborhood_latitude, 
                                                               m_neighborhood_longitude))

Latitude and longitude values of Delisle Road , Mumbai are 18.9932602, 72.8310115.


In [24]:
p_neighborhood_latitude = pune_data.loc[1, 'Latitude'] # neighborhood latitude value
p_neighborhood_longitude = pune_data.loc[1, 'Longitude'] # neighborhood longitude value

p_neighborhood_name = pune_data.loc[1, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(p_neighborhood_name, 
                                                               p_neighborhood_latitude, 
                                                               p_neighborhood_longitude))

Latitude and longitude values of Armament , Bavdhan , Sus , Pune are 18.537780750000003, 73.79634390000001.


In [25]:
# type your answer here

LIMIT = 10 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

m_url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    m_neighborhood_latitude, 
    m_neighborhood_longitude, 
    radius, 
    LIMIT)
m_url 

'https://api.foursquare.com/v2/venues/explore?&client_id=FR43BQ2IQEU02M0YQFTBEVKJIETDI3OGLAXLFRSNWPLK0X2M&client_secret=EJKACBJGC1VAZLLHZ4SVSJQ4A2UUD4MBGLJEM3I4VENF4SKQ&v=20180604&ll=18.9932602,72.8310115&radius=500&limit=10'

In [26]:
# type your answer here

LIMIT = 10 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

p_url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    p_neighborhood_latitude, 
    p_neighborhood_longitude, 
    radius, 
    LIMIT)
p_url

'https://api.foursquare.com/v2/venues/explore?&client_id=FR43BQ2IQEU02M0YQFTBEVKJIETDI3OGLAXLFRSNWPLK0X2M&client_secret=EJKACBJGC1VAZLLHZ4SVSJQ4A2UUD4MBGLJEM3I4VENF4SKQ&v=20180604&ll=18.537780750000003,73.79634390000001&radius=500&limit=10'

In [27]:
m_results = requests.get(m_url).json()
m_results

{'meta': {'code': 200, 'requestId': '5e3ea6d2618f43001b4ad3b7'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Parel',
  'headerFullLocation': 'Parel, Mumbai',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 3,
  'suggestedBounds': {'ne': {'lat': 18.997760204500008,
    'lng': 72.83576171930174},
   'sw': {'lat': 18.988760195499996, 'lng': 72.82626128069826}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '572306b6498e0709d28136cf',
       'name': 'Starbucks',
       'location': {'address': 'Marathon Futurex',
        'lat': 18.994788,
        'lng': 72.831275,
        'labeledLatLngs': [{'label': 'display',
          'lat': 18

In [28]:
p_results = requests.get(p_url).json()
p_results

{'meta': {'code': 200, 'requestId': '5e3ea6a114a126001bda02fe'},
 'response': {'headerLocation': 'Pune',
  'headerFullLocation': 'Pune',
  'headerLocationGranularity': 'city',
  'totalResults': 5,
  'suggestedBounds': {'ne': {'lat': 18.54228075450001,
    'lng': 73.80108130651435},
   'sw': {'lat': 18.533280745499997, 'lng': 73.79160649348567}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4e34f64f18a82fdd6571699c',
       'name': 'Pashan circle sabji market',
       'location': {'lat': 18.537706534903712,
        'lng': 73.79510147111766,
        'labeledLatLngs': [{'label': 'display',
          'lat': 18.537706534903712,
          'lng': 73.79510147111766}],
        'distance': 131,
        'cc': 'IN',
        'country': 'India',
        'formattedAddress': ['India'

In [29]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [30]:
m_venues = m_results['response']['groups'][0]['items']
    
m_nearby_venues = json_normalize(m_venues) # flatten JSON

# filter columns
m_filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
m_nearby_venues =m_nearby_venues.loc[:, m_filtered_columns]

# filter the category for each row
m_nearby_venues['venue.categories'] = m_nearby_venues.apply(get_category_type, axis=1)

# clean columns
m_nearby_venues.columns = [col.split(".")[-1] for col in m_nearby_venues.columns]

m_nearby_venues.head()

,name,categories,lat,lng
0,Starbucks,Coffee Shop,18.994788,72.831275
1,The True School of Music,Music Venue,18.995615,72.828023
2,Status,Indian Restaurant,18.997146,72.830546


In [31]:
p_venues = p_results['response']['groups'][0]['items']
    
p_nearby_venues = json_normalize(p_venues) # flatten JSON

# filter columns
p_filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
p_nearby_venues =p_nearby_venues.loc[:, p_filtered_columns]

# filter the category for each row
p_nearby_venues['venue.categories'] = p_nearby_venues.apply(get_category_type, axis=1)

# clean columns
p_nearby_venues.columns = [col.split(".")[-1] for col in p_nearby_venues.columns]

p_nearby_venues.head()

,name,categories,lat,lng
0,Pashan circle sabji market,Farmers Market,18.537707,73.795101
1,PASHAN CIRCLE BHAAJI MANDAI,Flea Market,18.537299,73.794286
2,LAKME SALON,Spa,18.535820,73.794678
3,avdut misal centre,Breakfast Spot,18.538790,73.793762
4,Tapkir Misalwale,Indian Restaurant,18.540549,73.796400


In [32]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [33]:
mumbai_venues = getNearbyVenues(names=mumbai_data['Neighbourhood'],
                                   latitudes=mumbai_data['Latitude'],
                                   longitudes=mumbai_data['Longitude']
                                  )

Malabar Hill , Mumbai
Delisle Road , Mumbai
Sewri , Mumbai
Dharavi Road , Dharavi , Mumbai
Matunga , Mumbai
Nehru Nagar  , Mumbai
New Prabhadevi Road , Prabhadevi , Mumbai
Haji Ali , Tulsiwadi , Mumbai
Bhandup East , Mumbai
Juhu , Mumbai
Bandra West , Mumbai
Andheri , Azad Nagar  , Mumbai
Santacruz Central , Santacruz(West) , Mumbai
Jogeshwari East , Mumbai
Andheri East , Nagardas Road , Mumbai
Powai Iit , Mumbai
Rajawadi , Mumbai
Mhada Colony , Mulund East , Mumbai
Kannamwar Nagar , Tagore Nagar , Mumbai
Barve Nagar , Mumbai
BARC , Mumbai
NITIE , Mumbai
Borivali , Mumbai
Borivali West , Mumbai
Chakala Midc , Mumbai
Anushakti Nagar , Mumbai
Ins Hamla , Kharodi , Mumbai
Seepz , Mumbai
Malad East , Rani Sati Marg , Mumbai
Kandivali East , Mumbai
Jogeshwari West , Oshiwara , Mumbai
Mandapeshwar , Mumbai


In [34]:
pune_venues = getNearbyVenues(names=pune_data['Neighbourhood'],
                                   latitudes=pune_data['Latitude'],
                                   longitudes=pune_data['Longitude']
                                  )

Khandala , Pune
Armament , Bavdhan , Sus , Pune
Srpf , Pune
Khadakwasla R.S. , Pune
C M E , Pune
Kasarwadi , Pune
Akurdi , Pune
Baner Gaon , N.I.A. , Pune
Ambegaon BK , Katraj , Pune
Anandnagar  , Pune
Warje , Pune
Pimple Gurav , Pune
Dasve Lavasa , Pune
Manjari BK , Manjari Farm , Pune


In [35]:
print(mumbai_venues.shape)
mumbai_venues.head()

(218, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malabar Hill , Mumbai",18.958162,72.803366,The Malabar Hill Club,18.954392,72.803510,Restaurant
1,"Malabar Hill , Mumbai",18.958162,72.803366,Priyadarshini Park,18.957515,72.799614,Park
2,"Malabar Hill , Mumbai",18.958162,72.803366,Santosh Sagar,18.961177,72.801425,Fast Food Restaurant
3,"Malabar Hill , Mumbai",18.958162,72.803366,Hanging Gardens,18.956328,72.805373,Park
4,"Malabar Hill , Mumbai",18.958162,72.803366,210°Veg Gourmet Bakery,18.961828,72.802137,Bakery


In [36]:
print(pune_venues.shape)
pune_venues.head()

(70, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Khandala , Pune",18.754847,73.376709,El Taj,18.758499,73.376364,Restaurant
1,"Khandala , Pune",18.754847,73.376709,khandala station,18.757955,73.376830,Train Station
2,"Khandala , Pune",18.754847,73.376709,Kamats Green House,18.757966,73.376793,Indian Restaurant
3,"Armament , Bavdhan , Sus , Pune",18.537781,73.796344,Pashan circle sabji market,18.537707,73.795101,Farmers Market
4,"Armament , Bavdhan , Sus , Pune",18.537781,73.796344,PASHAN CIRCLE BHAAJI MANDAI,18.537299,73.794286,Flea Market


In [37]:
mumbai_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Andheri , Azad Nagar , Mumbai",10,10,10,10,10,10
"Andheri East , Nagardas Road , Mumbai",10,10,10,10,10,10
"Anushakti Nagar , Mumbai",4,4,4,4,4,4
"Bandra West , Mumbai",10,10,10,10,10,10
"Barve Nagar , Mumbai",4,4,4,4,4,4
"Bhandup East , Mumbai",4,4,4,4,4,4
"Borivali , Mumbai",10,10,10,10,10,10
"Borivali West , Mumbai",10,10,10,10,10,10
"Chakala Midc , Mumbai",10,10,10,10,10,10


In [38]:
pune_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Akurdi , Pune",10,10,10,10,10,10
"Ambegaon BK , Katraj , Pune",2,2,2,2,2,2
"Anandnagar , Pune",10,10,10,10,10,10
"Armament , Bavdhan , Sus , Pune",5,5,5,5,5,5
"Baner Gaon , N.I.A. , Pune",10,10,10,10,10,10
"C M E , Pune",5,5,5,5,5,5
"Dasve Lavasa , Pune",4,4,4,4,4,4
"Kasarwadi , Pune",5,5,5,5,5,5
"Khadakwasla R.S. , Pune",1,1,1,1,1,1


In [39]:
print('There are {} uniques categories.'.format(len(mumbai_venues['Venue Category'].unique())))

There are 67 uniques categories.


In [40]:
print('There are {} uniques categories.'.format(len(pune_venues['Venue Category'].unique())))

There are 40 uniques categories.


In [41]:
# one hot encoding
mumbai_onehot = pd.get_dummies(mumbai_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
mumbai_onehot['Neighborhood'] = mumbai_venues['Neighborhood'] 

# move neighborhood column to the first column
m_fixed_columns = [mumbai_onehot.columns[-1]] + list(mumbai_onehot.columns[:-1])
mumbai_onehot = mumbai_onehot[m_fixed_columns]

mumbai_onehot.head()

,Women's Store,ATM,Asian Restaurant,Athletics & Sports,Bakery,Bar,Boat or Ferry,Burger Joint,Bus Station,Café,...,Scenic Lookout,Seafood Restaurant,Shopping Mall,Snack Place,South Indian Restaurant,Sports Bar,Steakhouse,Theater,Train Station,Vegetarian / Vegan Restaurant
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [42]:
# one hot encoding
pune_onehot = pd.get_dummies(pune_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
pune_onehot['Neighborhood'] = pune_venues['Neighborhood'] 

# move neighborhood column to the first column
p_fixed_columns = [pune_onehot.columns[-1]] + list(pune_onehot.columns[:-1])
pune_onehot = pune_onehot[p_fixed_columns]

pune_onehot.head()

,Neighborhood,ATM,Asian Restaurant,Bakery,Bar,Breakfast Spot,Burger Joint,Bus Station,Café,Coffee Shop,...,Resort,Restaurant,Seafood Restaurant,Snack Place,South Indian Restaurant,Spa,Surf Spot,Tea Room,Train,Train Station
0,"Khandala , Pune",0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,"Khandala , Pune",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,"Khandala , Pune",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Armament , Bavdhan , Sus , Pune",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Armament , Bavdhan , Sus , Pune",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [43]:
mumbai_onehot.shape

(218, 67)

In [44]:
pune_onehot.shape

(70, 41)

In [45]:
mumbai_grouped = mumbai_onehot.groupby('Neighborhood').mean().reset_index()
mumbai_grouped.head()

,Neighborhood,Women's Store,ATM,Asian Restaurant,Athletics & Sports,Bakery,Bar,Boat or Ferry,Burger Joint,Bus Station,...,Scenic Lookout,Seafood Restaurant,Shopping Mall,Snack Place,South Indian Restaurant,Sports Bar,Steakhouse,Theater,Train Station,Vegetarian / Vegan Restaurant
0,"Andheri , Azad Nagar , Mumbai",0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.1,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"Andheri East , Nagardas Road , Mumbai",0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.1,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"Anushakti Nagar , Mumbai",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,"Bandra West , Mumbai",0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"Barve Nagar , Mumbai",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [46]:
pune_grouped = pune_onehot.groupby('Neighborhood').mean().reset_index()
pune_grouped.head()

,Neighborhood,ATM,Asian Restaurant,Bakery,Bar,Breakfast Spot,Burger Joint,Bus Station,Café,Coffee Shop,...,Resort,Restaurant,Seafood Restaurant,Snack Place,South Indian Restaurant,Spa,Surf Spot,Tea Room,Train,Train Station
0,"Akurdi , Pune",0.0,0.1,0.1,0.0,0.0,0.0,0.0,0.2,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.1,0.0
1,"Ambegaon BK , Katraj , Pune",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"Anandnagar , Pune",0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.1,0.2,...,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0
3,"Armament , Bavdhan , Sus , Pune",0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0
4,"Baner Gaon , N.I.A. , Pune",0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.1,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [47]:
mumbai_grouped.shape

(30, 67)

In [48]:
pune_grouped.shape

(14, 41)

In [49]:
num_top_venues = 5

for hood in mumbai_grouped['Neighborhood']:
    print("----"+hood+"----")
    m_temp = mumbai_grouped[mumbai_grouped['Neighborhood'] == hood].T.reset_index()
    m_temp.columns = ['venue','freq']
    m_temp = m_temp.iloc[1:]
    m_temp['freq'] = m_temp['freq'].astype(float)
    m_temp = m_temp.round({'freq': 2})
    print(m_temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Andheri , Azad Nagar  , Mumbai----
                  venue  freq
0     Indian Restaurant   0.3
1  Fast Food Restaurant   0.2
2        Ice Cream Shop   0.1
3                Bakery   0.1
4          Burger Joint   0.1


----Andheri East , Nagardas Road , Mumbai----
                  venue  freq
0     Indian Restaurant   0.3
1  Fast Food Restaurant   0.2
2                Bakery   0.1
3            Restaurant   0.1
4          Burger Joint   0.1


----Anushakti Nagar , Mumbai----
                  venue  freq
0            Playground  0.25
1                  Lake  0.25
2  Fast Food Restaurant  0.25
3     Convenience Store  0.25
4         Jewelry Store  0.00


----Bandra West , Mumbai----
               venue  freq
0               Café   0.3
1      Deli / Bodega   0.1
2  Indian Restaurant   0.1
3               Park   0.1
4             Bakery   0.1


----Barve Nagar , Mumbai----
                venue  freq
0                Café  0.50
1  Light Rail Station  0.25
2          Food Truck  0.25
3 

In [50]:
num_top_venues = 5

for hood in pune_grouped['Neighborhood']:
    print("----"+hood+"----")
    p_temp = pune_grouped[pune_grouped['Neighborhood'] == hood].T.reset_index()
    p_temp.columns = ['venue','freq']
    p_temp = p_temp.iloc[1:]
    p_temp['freq'] = p_temp['freq'].astype(float)
    p_temp = p_temp.round({'freq': 2})
    print(p_temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Akurdi , Pune----
                       venue  freq
0                       Café   0.2
1       Fast Food Restaurant   0.2
2  Middle Eastern Restaurant   0.1
3                     Bakery   0.1
4                      Train   0.1


----Ambegaon BK , Katraj , Pune----
               venue  freq
0                Gym   0.5
1               Café   0.5
2                ATM   0.0
3             Resort   0.0
4  Mobile Phone Shop   0.0


----Anandnagar  , Pune----
                venue  freq
0         Coffee Shop   0.2
1   Indian Restaurant   0.2
2        Burger Joint   0.1
3  Italian Restaurant   0.1
4                Café   0.1


----Armament , Bavdhan , Sus , Pune----
               venue  freq
0     Farmers Market   0.2
1     Breakfast Spot   0.2
2                Spa   0.2
3  Indian Restaurant   0.2
4        Flea Market   0.2


----Baner Gaon , N.I.A. , Pune----
                             venue  freq
0                Indian Restaurant   0.3
1                           Lounge   0.2
2      

In [51]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### Top 5 venues of Neibourhoods of Mumbai

In [52]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
m_neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
m_neighborhoods_venues_sorted['Neighborhood'] = mumbai_grouped['Neighborhood']

for ind in np.arange(mumbai_grouped.shape[0]):
    m_neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(mumbai_grouped.iloc[ind, :], num_top_venues)

m_neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,"Andheri , Azad Nagar , Mumbai",Indian Restaurant,Fast Food Restaurant,Food Court,Ice Cream Shop,Bakery
1,"Andheri East , Nagardas Road , Mumbai",Indian Restaurant,Fast Food Restaurant,Food Court,Bakery,Burger Joint
2,"Anushakti Nagar , Mumbai",Fast Food Restaurant,Lake,Playground,Convenience Store,Dance Studio
3,"Bandra West , Mumbai",Café,Park,French Restaurant,Indian Restaurant,Deli / Bodega
4,"Barve Nagar , Mumbai",Café,Light Rail Station,Food Truck,Vegetarian / Vegan Restaurant,Dance Studio


#### Top 5 venues of Neibourhoods of Pune

In [53]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
p_neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
p_neighborhoods_venues_sorted['Neighborhood'] = pune_grouped['Neighborhood']

for ind in np.arange(pune_grouped.shape[0]):
    p_neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(pune_grouped.iloc[ind, :], num_top_venues)

p_neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,"Akurdi , Pune",Café,Fast Food Restaurant,Train,Tea Room,Asian Restaurant
1,"Ambegaon BK , Katraj , Pune",Gym,Café,Train Station,Dessert Shop,Indian Restaurant
2,"Anandnagar , Pune",Indian Restaurant,Coffee Shop,Burger Joint,Dessert Shop,Italian Restaurant
3,"Armament , Bavdhan , Sus , Pune",Indian Restaurant,Spa,Flea Market,Breakfast Spot,Farmers Market
4,"Baner Gaon , N.I.A. , Pune",Indian Restaurant,Lounge,Juice Bar,Malay Restaurant,Breakfast Spot


#### K means cluster analysis of Mumbai

In [54]:
# set number of clusters
kclusters = 5

mumbai_grouped_clustering = mumbai_grouped.drop('Neighborhood',1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(mumbai_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 0, 0, 4, 4, 1, 1, 4], dtype=int32)

In [55]:
# add clustering labels
m_neighborhoods_venues_sorted.insert(0, 'Cluster Labels',kmeans.labels_,allow_duplicates=True)

mumbai_merged = mumbai_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
mumbai_merged = mumbai_merged.join(m_neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')
mumbai_merged.dropna(inplace=True)
mumbai_merged.head() # check the last columns!

,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,"Malabar Hill , Mumbai",18.958162,72.803366,1.0,Park,Bakery,Restaurant,Fast Food Restaurant,Gym
1,"Delisle Road , Mumbai",18.993260,72.831012,4.0,Coffee Shop,Indian Restaurant,Music Venue,Vegetarian / Vegan Restaurant,Donut Shop
2,"Sewri , Mumbai",18.998780,72.854422,4.0,Harbor / Marina,Bus Station,Flea Market,Train Station,French Restaurant
4,"Matunga , Mumbai",19.027436,72.850147,4.0,Indian Restaurant,Vegetarian / Vegan Restaurant,Bar,Train Station,Flower Shop
5,"Nehru Nagar , Mumbai",19.001048,72.819427,4.0,Ice Cream Shop,Bakery,Indian Restaurant,Seafood Restaurant,Dessert Shop


In [56]:
# create map
map_clusters_mumbai = folium.Map(location=[m_latitude, m_longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(mumbai_merged['Latitude'], mumbai_merged['Longitude'], mumbai_merged['Neighbourhood'], mumbai_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        fill=True,
        fill_opacity=0.7).add_to(map_clusters_mumbai)
       
map_clusters_mumbai

A piece of important information this map provides is that many neighborhoods in Mumbai are of similar nature concerning the venues they have around, indicated by the cluster marked in blue.

In [57]:
# set number of clusters
kclusters = 5

pune_grouped_clustering = pune_grouped.drop('Neighborhood',1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(pune_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 0, 3, 3, 3, 2, 2, 3, 4, 3], dtype=int32)

In [58]:
# add clustering labels
p_neighborhoods_venues_sorted.insert(0, 'Cluster Labels',kmeans.labels_,allow_duplicates=True)

pune_merged = pune_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
pune_merged = pune_merged.join(p_neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')
pune_merged.dropna(inplace=True)
pune_merged.head() # check the last columns!

,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,"Khandala , Pune",18.754847,73.376709,3,Train Station,Restaurant,Indian Restaurant,Bakery,Dessert Shop
1,"Armament , Bavdhan , Sus , Pune",18.537781,73.796344,3,Indian Restaurant,Spa,Flea Market,Breakfast Spot,Farmers Market
2,"Srpf , Pune",18.502487,73.905953,0,Gym,Breakfast Spot,Electronics Store,Park,Train Station
3,"Khadakwasla R.S. , Pune",18.439243,73.775766,4,Indian Restaurant,Train Station,Train,Gym,Food Truck
4,"C M E , Pune",18.555492,73.952473,2,North Indian Restaurant,Coffee Shop,Pub,Department Store,Gym


In [59]:
# create map
map_clusters_pune = folium.Map(location=[p_latitude, p_longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(pune_merged['Latitude'], pune_merged['Longitude'], pune_merged['Neighbourhood'], pune_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        fill=True,
        fill_opacity=0.7).add_to(map_clusters_pune)
       
map_clusters_pune

A piece of important information this map provides is that many neighborhoods in Pune are of similar nature concerning the venues they have around, indicated by the cluster marked in blue but one important point is they are far away from each other compared to Mumbai.

### Cluster 0

In [60]:
mumbai_merged.loc[mumbai_merged['Cluster Labels'] == 0, mumbai_merged.columns[[0] + list(range(4, mumbai_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
10,"Bandra West , Mumbai",Café,Park,French Restaurant,Indian Restaurant,Deli / Bodega
15,"Powai Iit , Mumbai",Dance Studio,Café,Hotel Bar,Department Store,Indian Restaurant
19,"Barve Nagar , Mumbai",Café,Light Rail Station,Food Truck,Vegetarian / Vegan Restaurant,Dance Studio


Cafe's in this Neighbourhoods seems to be more popular

### Cluster 1

In [71]:
mumbai_merged.loc[mumbai_merged['Cluster Labels'] == 1, mumbai_merged.columns[[0] + list(range(4, mumbai_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,"Malabar Hill , Mumbai",Park,Bakery,Restaurant,Fast Food Restaurant,Gym
11,"Andheri , Azad Nagar , Mumbai",Indian Restaurant,Fast Food Restaurant,Food Court,Ice Cream Shop,Bakery
14,"Andheri East , Nagardas Road , Mumbai",Indian Restaurant,Fast Food Restaurant,Food Court,Bakery,Burger Joint
16,"Rajawadi , Mumbai",Fast Food Restaurant,Train Station,Bar,Juice Bar,Chinese Restaurant
18,"Kannamwar Nagar , Tagore Nagar , Mumbai",Comedy Club,Sake Bar,ATM,Bakery,Indian Restaurant
23,"Borivali West , Mumbai",Indian Restaurant,Chinese Restaurant,Vegetarian / Vegan Restaurant,Snack Place,Fast Food Restaurant
24,"Chakala Midc , Mumbai",Hotel,Fast Food Restaurant,Café,Pizza Place,Indian Restaurant
25,"Anushakti Nagar , Mumbai",Fast Food Restaurant,Lake,Playground,Convenience Store,Dance Studio
29,"Kandivali East , Mumbai",Fast Food Restaurant,Food & Drink Shop,Sports Bar,Lounge,Pizza Place


Neibourhoods belonging to this cluster seems to be popular for Indian restaurants, Fast food, Juice centers, Ice cream shops, and many more food items. 

### Cluster 2

In [62]:
mumbai_merged.loc[mumbai_merged['Cluster Labels'] == 2, mumbai_merged.columns[[0] + list(range(4, mumbai_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
28,"Malad East , Rani Sati Marg , Mumbai",Vegetarian / Vegan Restaurant,Shopping Mall,Fast Food Restaurant,Cricket Ground,Dance Studio


Very few Neibourhoods belong to this cluster may be more interested in eating vegetarian or vegan food

### Cluster 3

In [63]:
mumbai_merged.loc[mumbai_merged['Cluster Labels'] == 3, mumbai_merged.columns[[0] + list(range(4, mumbai_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
13,"Jogeshwari East , Mumbai",Moving Target,Mattress Store,Vegetarian / Vegan Restaurant,Donut Shop,Dance Studio
26,"Ins Hamla , Kharodi , Mumbai",Moving Target,Boat or Ferry,Vegetarian / Vegan Restaurant,Fast Food Restaurant,Dance Studio


This cluster seems to be popular for ferry rides and Dance studio.

### Cluster 4

In [64]:
mumbai_merged.loc[mumbai_merged['Cluster Labels'] == 4, mumbai_merged.columns[[0] + list(range(4, mumbai_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
1,"Delisle Road , Mumbai",Coffee Shop,Indian Restaurant,Music Venue,Vegetarian / Vegan Restaurant,Donut Shop
2,"Sewri , Mumbai",Harbor / Marina,Bus Station,Flea Market,Train Station,French Restaurant
4,"Matunga , Mumbai",Indian Restaurant,Vegetarian / Vegan Restaurant,Bar,Train Station,Flower Shop
5,"Nehru Nagar , Mumbai",Ice Cream Shop,Bakery,Indian Restaurant,Seafood Restaurant,Dessert Shop
6,"New Prabhadevi Road , Prabhadevi , Mumbai",Bakery,Women's Store,Restaurant,Coffee Shop,Indian Restaurant
7,"Haji Ali , Tulsiwadi , Mumbai",Bar,Vegetarian / Vegan Restaurant,Deli / Bodega,Juice Bar,Italian Restaurant
8,"Bhandup East , Mumbai",Ice Cream Shop,Train Station,Asian Restaurant,Flea Market,Deli / Bodega
9,"Juhu , Mumbai",Lounge,Café,Theater,Hotel,Cocktail Bar
12,"Santacruz Central , Santacruz(West) , Mumbai",Indian Restaurant,Women's Store,Snack Place,Market,Jewelry Store
17,"Mhada Colony , Mulund East , Mumbai",Indian Restaurant,Vegetarian / Vegan Restaurant,Ice Cream Shop,Pizza Place,Playground


Neibourhoods in this cluster seems to be perfect for living as all kinds required thing are there.

Overall Mumbai seems to be made for foodie people as almost all clusters contains different types of food i.e. Chinese, Italian, Street foods, Fast foods etc.

So at this stage we can conclude that Foodie people will like Mumbai.

### Cluster 0

In [65]:
pune_merged.loc[pune_merged['Cluster Labels'] == 0, pune_merged.columns[[0] + list(range(4, pune_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
2,"Srpf , Pune",Gym,Breakfast Spot,Electronics Store,Park,Train Station
8,"Ambegaon BK , Katraj , Pune",Gym,Café,Train Station,Dessert Shop,Indian Restaurant


This cluster belongs to very less Neibourhoods with 1st most common venue as Gym. So may be Health focussed people will like this Neibourhoods.

### Cluster 1

In [66]:
pune_merged.loc[pune_merged['Cluster Labels'] == 1, pune_merged.columns[[0] + list(range(4, pune_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
13,"Manjari BK , Manjari Farm , Pune",ATM,Train,Indian Restaurant,Gym,Food Truck


Very few Neibourhoods belong to this cluster with again one of the common venue as Gym and Indian Restaurant.

### Cluster 2

In [67]:
pune_merged.loc[pune_merged['Cluster Labels'] == 2, pune_merged.columns[[0] + list(range(4, pune_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
4,"C M E , Pune",North Indian Restaurant,Coffee Shop,Pub,Department Store,Gym
11,"Pimple Gurav , Pune",Department Store,Food Truck,Fast Food Restaurant,Snack Place,Restaurant
12,"Dasve Lavasa , Pune",Surf Spot,Bakery,Resort,Music Venue,Train Station


Neighbourhoods belonging to this cluster may be living a simple lifestyle as Deparmental store is the 1st most common venue.

### Cluster 3

In [68]:
pune_merged.loc[pune_merged['Cluster Labels'] == 3, pune_merged.columns[[0] + list(range(4, pune_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,"Khandala , Pune",Train Station,Restaurant,Indian Restaurant,Bakery,Dessert Shop
1,"Armament , Bavdhan , Sus , Pune",Indian Restaurant,Spa,Flea Market,Breakfast Spot,Farmers Market
5,"Kasarwadi , Pune",Indian Restaurant,Platform,Mobile Phone Shop,Fast Food Restaurant,Bus Station
6,"Akurdi , Pune",Café,Fast Food Restaurant,Train,Tea Room,Asian Restaurant
7,"Baner Gaon , N.I.A. , Pune",Indian Restaurant,Lounge,Juice Bar,Malay Restaurant,Breakfast Spot
9,"Anandnagar , Pune",Indian Restaurant,Coffee Shop,Burger Joint,Dessert Shop,Italian Restaurant
10,"Warje , Pune",Indian Restaurant,Bar,Seafood Restaurant,Department Store,Gym


So many Indian Restaurants are belonging to this cluster, suggests that people living here may be try to avoid fast food. 

### Cluster 4

In [69]:
pune_merged.loc[pune_merged['Cluster Labels'] == 4, pune_merged.columns[[0] + list(range(4, pune_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
3,"Khadakwasla R.S. , Pune",Indian Restaurant,Train Station,Train,Gym,Food Truck


Very few Neighbourhoods belongs to this cluster. The most common venues here are again Indian Restaurant, Gym, Train/Railway station. 


Almost all clusters of Pune City suggesting that peaple living here may have a simple lifestyle.

## References

### Website
   https://medium.com/towards-artificial-intelligence/mumbai-versus-delhi-what-is-your-choice-let-data-decide-69270c832dbc

## Findings & Conclusion

In this project, we have attempted to load the dataset for two of Maharashtra’s prime cities and have tried to analyze the neighborhood regions in these cities based on the type of popular and top venues they have. I have clustered the neighborhoods based on the most common top venues in each of the neighborhood. Our intention with this project was to analyze and understand the difference in the type of life in these cities, which can offer decision points for anybody who is considering to settle in either of the cities and can get a peek into what type of experience and facilities he will be provided with.
Given our cluster information for both Mumbai and Pune, we see that Mumbai and its neighbourhoods are a great place for a foodie. There are a lot of restaurants, cafes, bars, etc in Mumbai neighbourhoods. Also due to the proximity of Mumbai to the seashore, Mumbai neighborhoods offer for harbors, seafood, boat, and ferry rides. On the other hand, we see how dissimilar life in Pune neighbourhoods would be compared to Mumbai neighbourhoods. Pune neighborhoods like a simple lifestyles Indian restaurants, Gyms, Breakfast spots seems to be more there. Mumbai, on the other hand, is great for international visitors, expats, etc, because of the variety and types of food outlets it has. People living in Pune may be more health consious, as departmental stores are more there they may be generally prefers homemade food. Even there choice for eating outside food is again Indian restaurants.


Thus with this project, we have analyzed the kind of life each of these cities has to offer based on the popular venues in their neighborhood.

So, at the end I will suggest that if you are foodie, then Mumbai is the choice for you!!

Another important aspect the study reveals is that the categories of venues Mumbai offers are far too many compared to Pune, even the area wise Pune is big City. This means that Pune becomes restrictive in terms of variety and convenience. With the data, we have studied Mumbai wins this debate of cities!


## So Mumbai is the perfect City!!